# Preprocess the data

Create the following matrix:

*   Rows: UserID's
*   Columns: MovieID's
*   Cell: The rating of the movie


In [59]:
# !pip install pyspark
# !apt install openjdk-8-jdk-headless -qq

# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.decomposition import SparsePCA

import scipy.sparse as sparse

from datetime import datetime
import os

import json

# import pyspark
# from pyspark.sql import *
# from pyspark.sql.types import *
# from pyspark.sql.functions import *
# from pyspark import SparkContext, SparkConf
# from pyspark.mllib.linalg.distributed import RowMatrix
# from pyspark.mllib.linalg import Vectors
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.feature import StandardScaler

In [61]:
# # create the session
# conf = SparkConf().set("spark.ui.port", "4050")

# # create the context
# sc = pyspark.SparkContext(conf=conf)
# spark = SparkSession.builder.getOrCreate()

In [62]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [63]:
dateparse = lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')

files_path = '/content/drive/MyDrive/CSE547_Final_Project/ml-100k/ratings_split/iter_5'
# files_path = '/content/drive/MyDrive/CSE547_Final_Project/ml-100k/train_test_v2/rawdata_split/iter_5'
# files_path = '/content/drive/MyDrive/CSE547_Final_Project/ml-25m'
ratings_file = os.path.join('/content/drive/MyDrive/CSE547_Final_Project/ml-100k', "ratings.csv")
# ratings_file = os.path.join(files_path, "train_iter5.csv")
# movies_file = os.path.join(files_path, "movies.csv")
# user_movie_ratings_matrix = os.path.join(files_path, "user_movie_ratings_matrix.csv")
user_movie_ratings_matrix = os.path.join(files_path, "user_movie_ratings_matrix.csv")
user_to_idx_file = os.path.join(files_path, "user_to_idx.json")
movie_to_idx_file = os.path.join(files_path, "movie_to_idx.json")

train_file_name = os.path.join(files_path, 'ratings_train.csv')
test_file_name = os.path.join(files_path, 'ratings_test.csv')
validation_file_name = os.path.join(files_path, 'ratings_validation.csv')

ratings_df = pd.read_csv(   ratings_file, 
                            parse_dates=['timestamp'], 
                            date_parser=dateparse)
# ratings_df = pd.read_csv(ratings_file)
# movies_df = pd.read_csv(movies_file)

In [64]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51


In [65]:
ratings_df.drop(columns=['timestamp'], inplace=True)

In [66]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [67]:
X, y = ratings_df.iloc[:, :-1], ratings_df.iloc[:, -1]

In [68]:
# X, X_test, y, y_test = train_test_split(X, y, test_size=0.1, random_state=37)
X, X_test, y, y_test = train_test_split(X, y, test_size=0.1)

In [69]:
# X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.11, random_state=72)
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.11)

In [70]:
pd.concat([X_train, y_train], axis=1).to_csv(train_file_name, index=False)
pd.concat([X_validate, y_validate], axis=1).to_csv(validation_file_name, index=False)
pd.concat([X_test, y_test], axis=1).to_csv(test_file_name, index=False)

In [71]:
print(X_train.shape, X_validate.shape, X_test.shape)
print(y_train.shape, y_validate.shape, y_test.shape)

(80769, 2) (9983, 2) (10084, 2)
(80769,) (9983,) (10084,)


In [72]:
X_train.head()

,userId,movieId
78146,484,6870
6778,45,3408
68132,438,8869
89080,577,780
91625,594,2716


In [73]:
y_train.head()

78146    4.0
6778     4.0
68132    2.0
89080    3.0
91625    4.0
Name: rating, dtype: float64

In [74]:
def create_utility_matrix(X, y, formatizer = {'userId':0, 'movieId': 1}):
    itemField = formatizer['movieId']
    userField = formatizer['userId']
    userList = X.iloc[:,userField].tolist()
    itemList = X.iloc[:,itemField].tolist()
    valueList = y.tolist()

    users = list(set(X.iloc[:,userField]))
    items = list(set(X.iloc[:,itemField]))

    users_index = {users[i]: i for i in range(len(users))}
    items_index = {items[i]: i for i in range(len(items))}

    USER_LEN, ITEM_LEN = len(users_index), len(items_index)

    # csr = np.zeros((USER_LEN, ITEM_LEN + 1))
    csr = np.zeros((USER_LEN, ITEM_LEN))
    for index, row in X.iterrows():
        # csr[
        #         users_index[int(row[userField])],
        #         0
        #     ] = row[userField]
        csr[
                users_index[int(row[userField])],
                items_index[int(row[itemField])]
            ] = y[index]
    
    return csr, users_index, items_index

In [75]:
X_train_u_matrix, X_train_user_idx, X_train_item_idx = create_utility_matrix(X_train, y_train)
X_train_u_matrix_df = pd.DataFrame(X_train_u_matrix)
# X_train_u_matrix_df.rename(columns = {0: 'userId',}, inplace = True)
X_train_u_matrix_df.to_csv(user_movie_ratings_matrix, index=False)

In [76]:
with open(user_to_idx_file, "w") as f1: 
    json.dump(X_train_user_idx, f1, indent=4)

In [77]:
with open(movie_to_idx_file, "w") as f2: 
    json.dump(X_train_item_idx, f2, indent=4)

In [78]:
# X_train_u_matrix_df.iloc[X_train_user_idx[463], X_train_item_idx[8622]]

In [79]:
X_train_u_matrix_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,8911,8912,8913,8914,8915,8916,8917,8918,8919,8920,8921,8922,8923,8924,8925,8926,8927,8928,8929,8930,8931,8932,8933,8934,8935,8936,8937,8938,8939,8940,8941,8942,8943,8944,8945,8946,8947,8948,8949,8950
0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Spark Pivot Table

In [80]:
# trainset = spark.read.csv(train_file_name, header=True, inferSchema=True)

In [81]:
# trainset.head()

In [82]:
# distinctValuesDF = trainset.select(trainset["movieId"]).distinct()

In [83]:
# distinctValuesDF.

In [84]:
# spark.conf.set('spark.sql.pivotMaxValues', distinctValuesDF.count())

In [85]:
# ratings_pivot = trainset.groupBy("userId")\
#                         .pivot("movieId")\
#                         .agg(expr("coalesce(first(rating), 3)")
#                         .cast("double"))

In [86]:
# ratings_pivot_df = ratings_pivot.toPandas()

In [87]:
# ratings_pivot_df